In [1]:
#!pip install dateparser

In [4]:
from openai import OpenAI
import os

# Inicializar cliente con tu API Key (usando variable de entorno)
client = OpenAI(api_key=)

KeyError: 'OPENAI_API_KEY'

🧪 Verificación rápida

In [ ]:
import os
print("OPENAI_API_KEY" in os.environ)

In [ ]:
import os
print(os.environ.get("OPENAI_API_KEY"))

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "Dame un número del 1 al 5 según tu nivel de preocupación fiscal"}
    ],
    temperature=0,
    max_tokens=5
)

print("Respuesta del modelo:", response.choices[0].message.content.strip())

# ----------------------------------------------
# 1. Cargar librerías
# ----------------------------------------------

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import openai
import time

# ----------------------------------------------
# 2. Cargar tu archivo original
# ----------------------------------------------

In [ ]:
# Asegúrate que tu archivo esté en el mismo directorio
df = pd.read_csv("sentiment_data.csv")  # Debe contener una columna 'text' y 'date'

# Inicializar columnas para los resultados
df["fiscal_risk_score"] = None        # Puntaje del 1 al 5
df["risk_index"] = None               # Normalizado a escala 0.0–1.0

In [ ]:
# Asegurar columnas necesarias
assert 'text' in df.columns, "Falta columna 'text' con los párrafos"
assert 'date' in df.columns, "Falta columna 'date' con la fecha del documento"

In [ ]:
df.head(20)

In [ ]:
import pandas as pd
import dateparser

def parse_spanish_dates(df, column_name):
    """
    Convierte una columna de fechas en español a datetime.
    
    Parámetros:
        df (pd.DataFrame): El DataFrame con la columna a procesar.
        column_name (str): El nombre de la columna con fechas en español.

    Retorna:
        pd.DataFrame: El DataFrame original con la columna convertida a datetime.
    """
    df[column_name] = df[column_name].apply(
        lambda x: dateparser.parse(x, languages=['es']) if pd.notnull(x) else None
    )
    return df


In [ ]:
# Aplica la función
df = parse_spanish_dates(df, 'date')

# Verifica
print(df[['date']].head(5))

In [ ]:
df

# ----------------------------------------------
# 3. Función para obtener score desde GPT (1–5)
# ----------------------------------------------

In [ ]:
context = """
Sabemos que desde aproximadamente 2016 el manejo de las finanzas públicas ha mostrado signos crecientes de deterioro. 
La pérdida de disciplina fiscal, la falta de transparencia y el relajamiento de las reglas fiscales han sido temas 
recurrentes en los informes del Consejo. A ello se suma el impacto de la inestabilidad política —con frecuentes cambios 
ministeriales— sobre la capacidad institucional para llevar una política fiscal prudente y sostenible. En este contexto, 
el Consejo ha venido alertando con más frecuencia y firmeza sobre el incumplimiento de metas fiscales, el deterioro del 
balance público, y los riesgos de un endeudamiento creciente y potencialmente insostenible.

Criterios comunes en los informes del Consejo Fiscal (según categoría):

1. Cumplimiento y disciplina fiscal:
(disciplina fiscal, incumplimiento de metas fiscales, relajamiento de reglas fiscales, uso inadecuado del gasto público, desviación del déficit fiscal, deterioro del marco fiscal, flexibilización sin justificación, política fiscal procíclica)

2. Riesgo y sostenibilidad:
(riesgo fiscal, riesgo de sostenibilidad de la deuda, endeudamiento excesivo, dependencia de ingresos extraordinarios, vulnerabilidad fiscal estructural, uso de medidas transitorias o no permanentes, incertidumbre macrofiscal)

3. Gobernanza e institucionalidad:
(transparencia fiscal, calidad del gasto público, incertidumbre institucional, falta de planificación multianual, cambios frecuentes en autoridades económicas, debilitamiento institucional, independencia fiscal comprometida, ausencia de reforma estructural)
"""

    """
    Evalúa un párrafo de un informe fiscal utilizando GPT-4o y asigna un score del 1 al 5
    según el nivel de preocupación o alarma fiscal expresado.

    Returns:
        - int (1 a 5): Score de riesgo fiscal asignado por el modelo
        - None: En caso de error o respuesta inválida
    """

Interpretación:
- 1 = Sin preocupación fiscal (consolidación fiscal sostenida, cumplimiento de metas, transparencia fiscal, calidad del gasto público, planificación multianual)
- 2 = Ligera preocupación (observaciones menores, advertencias marginales, riesgo fiscal potencial, uso de medidas transitorias o ingresos extraordinarios)
- 3 = Neutral (descripción técnica sin énfasis positivo ni negativo, gestión fiscal dentro del marco previsto)
- 4 = Alta preocupación (incumplimiento de metas, flexibilización sin justificación, incertidumbre macrofiscal, debilitamiento institucional)
- 5 = Alarma fiscal (críticas severas, endeudamiento excesivo, deterioro del marco fiscal, independencia fiscal comprometida)

In [ ]:
def get_llm_score(text):
    prompt = f"""
Eres un analista técnico del Consejo Fiscal de Perú. Evalúa el siguiente párrafo extraído de un informe técnico del Consejo Fiscal (CF), donde se emite una opinión sobre el desempeño fiscal del Ministerio de Economía y Finanzas (MEF) en cuanto al cumplimiento de las metas fiscales.

Tu tarea es asignar un **puntaje del 1 al 5** según el **nivel de preocupación o alerta fiscal expresado en el texto**.

Interpretación:
- 1 = Sin preocupación fiscal (cumplimiento de metas, transparencia fiscal, planificación multianual)
- 2 = Ligera preocupación (riesgo fiscal potencial, desviación del déficit, dependencia de ingresos extraordinarios)
- 3 = Neutral (descripción técnica, gestión dentro del marco, sin juicio valorativo)
- 4 = Alta preocupación (incumplimiento de metas, relajamiento fiscal, incertidumbre macroeconómica)
- 5 = Alarma fiscal (críticas severas, riesgo de sostenibilidad de la deuda, independencia fiscal comprometida)

Devuelve solo un número del 1 al 5.

Texto:
\"\"\"{text}\"\"\"
""".strip()

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=5,
        )
        result = response.choices[0].message.content.strip()
        if result in {'1', '2', '3', '4', '5'}:
            return int(result)
        else:
            print(f"⚠️ Respuesta inesperada: {result}")
            return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None


# ----------------------------------------------
# 4. Ejecutar clasificación (con backup y control de crédito)
# ----------------------------------------------

In [ ]:
import openai
print(openai.__version__)


In [ ]:
from datetime import datetime

start = time.time()

for i, row in df.iterrows():
    if pd.notna(row["fiscal_risk_score"]):
        continue  # Ya procesado

    score = get_llm_score(row["text"])

    # Guardar score
    df.at[i, "fiscal_risk_score"] = score
    df.at[i, "risk_index"] = score / 5 if score else None

    # Backup cada 10 filas
    if i % 10 == 0:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        df.to_csv(f"backup_fiscal_risk_{timestamp}.csv", index=False)
        print(f"💾 Backup guardado en fila {i} - {timestamp}")

    time.sleep(1.2)  # Control para no sobrecargar

end = time.time()
print(f"✅ Proceso completado en {round((end - start)/60, 2)} minutos.")



# Guardar resultados completos

In [ ]:
df.to_csv("df_fiscal_scored.csv", index=False)

# Fleeting inspect

In [ ]:
df["fiscal_risk_score"].value_counts(dropna=False).sort_index()

In [ ]:
df["risk_index"].describe()

# ----------------------------------------------
# 5. Agregar score promedio por fecha
# ----------------------------------------------

In [ ]:
df_daily = df.groupby('date')['risk_index'].mean().reset_index()

# ----------------------------------------------
# 6. Visualizar índice fiscal
# ----------------------------------------------

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(df_daily['date'], df_daily['risk_index'], marker='o', linestyle='-', color='crimson')
plt.title("Índice de Riesgo Fiscal (GPT)")
plt.xlabel("Fecha")
plt.ylabel("Índice normalizado (0–1)")
plt.ylim(0, 1)
plt.grid(True)
plt.tight_layout()
plt.savefig("indice_riesgo_fiscal_llm.png", dpi=300)
plt.show()